# Initializing the environment and clusters

In [2]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [3]:
%%px --local
import sys
sys.path.append("/home/sac086/wisdm_model_personalization/")
import warnings
import os
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit, ShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode
from collections import Counter
import time

In [125]:
%%px --local
from imp import reload
reload(wisdm)

<module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[0:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[1:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[2:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[3:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

...

Out[27:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[28:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[29:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[30:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

Out[31:7]: <module 'wisdm.wisdm' from '/home/sac086/wisdm_model_personalization/wisdm/wisdm.py'>

# Experiment #1

In [134]:
%%px --local
experiment_name = "experiment_09-01_train_v1_random/"

description = '''
Training and calibrating probability estimations on the WISDM v1.1 dataset. 
Probability estimation was done with stream sampling where those samples whose most probable class label
was did not exceed some threshold (where the threshold is some amount greater than the uniform probability for the class) were
actively labeled.  Testing on WISDM v1.1 with one participant held out'''

experiment_output_path = "/home/sac086/wisdm_model_personalization/results/" + experiment_name

In [111]:
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)
with open(experiment_output_path + "README.md", "w") as fOut:
    fOut.write(description)

In [123]:
command = """wisdm.pipeline1('1', '%s', user_ids)""" % experiment_output_path

In [136]:
wisdm.set_data(version="1")

In [127]:
# divide the user ids up among the different cores
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

start = time.time()
dview.block = True
results = dview.execute(command)
finish = time.time()
print("Finished all models in %s minutes" % ((finish - start) / 60.))

Finished all models in 43.835192906856534 minutes


### Experiment #1 Results


In [137]:
results_df = viz.get_results_dataframe(experiment_output_path)

Loading results from /home/sac086/wisdm_model_personalization/results/experiment_09-01_train_v1_random/
36 users loaded, 0 users excluded
These users may not have had enough data:
[]


In [139]:
viz.set_data(version="1")
fig = viz.accuracy_means_by_training_size()
iplot(fig)

In [143]:
fig = viz.plotUserBests()
fig.layout['title'] = "# of users whose best performance came from a particular approach"
iplot(fig)

# Experiment 2

In [145]:
%%px --local
experiment_name = "experiment_09-01_train_v2_random/"

description = '''
Training and calibrating probability estimations on the WISDM v2.0 dataset. 
Probability estimation was done with stream sampling where those samples whose most probable class label
was did not exceed some threshold (where the threshold is some amount greater than the uniform probability for the class) were
actively labeled.  Testing on WISDM v2.0 with one participant held out'''

experiment_output_path = "/home/sac086/wisdm_model_personalization/results/" + experiment_name

In [129]:
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)
with open(experiment_output_path + "README.md", "w") as fOut:
    fOut.write(description)

In [130]:
command = """wisdm.pipeline1('2', '%s', user_ids)""" % experiment_output_path

In [146]:
wisdm.set_data(version="2")

In [132]:
# divide the user ids up among the different cores
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

start = time.time()
dview.block = True
results = dview.execute(command)
finish = time.time()
print("Finished all models in %s minutes" % ((finish - start) / 60.))

Finished all models in 20.96520482301712 minutes


### Experiment #2 Results

In [147]:
results_df = viz.get_results_dataframe(experiment_output_path)

Loading results from /home/sac086/wisdm_model_personalization/results/experiment_09-01_train_v2_random/
26 users loaded, 28 users excluded
These users may not have had enough data:
['1097', '1205', '1247', '1269', '1276', '1277', '1280', '1480', '1491', '1511', '1518', '1531', '1554', '1679', '1683', '1696', '1723', '1724', '1726', '1745', '1750', '1757', '1761', '1763', '1797', '1802', '1813', '1814']


In [148]:
viz.set_data(version="2")
fig = viz.accuracy_means_by_training_size()
iplot(fig)

In [149]:
fig = viz.plotUserBests()
fig.layout['title'] = "# of users whose best performance came from a particular approach"
iplot(fig)

# Experiment #3
In this experiment we compare differing amounts of personal data selected at random to combine with the impersonal 
training set as well as a cluster of the impersonal training set

In [150]:
%%px --local
experiment_name = "experiment_08-31_train_v1_test_v2_random/"

description = '''
Training and calibrating probability estimations on the WISDM v1.1 dataset. 
Probability estimation was done with stream sampling where those samples whose most probable class label
was did not exceed some threshold (where the threshold is some amount greater than the uniform probability for the class) were
actively labeled.  Testing on WISDM 2.0 with one participant held out'''

experiment_output_path = "/home/sac086/wisdm_model_personalization/results/" + experiment_name

In [5]:
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)
with open(experiment_output_path + "README.md", "w") as fOut:
    fOut.write(description)

In [5]:
command = """wisdm.pipeline2('%s', user_ids)""" % experiment_output_path

In [6]:
wisdm.set_data(version="2", make_compatible=True)

In [7]:
# divide the user ids up among the different cores
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

start = time.time()
dview.block = True
results = dview.execute(command)
finish = time.time()
print("Finished all models in %s minutes" % ((finish - start) / 60.))

Finished all models in 38.365303961435956 minutes


### Visualize Results for Experiment #3

In [78]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *

In [79]:
from imp import reload

In [80]:
from wisdm import viz

In [103]:
reload(viz)

<module 'wisdm.viz' from '/home/sac086/wisdm_model_personalization/wisdm/viz.py'>

In [151]:
results_df = viz.get_results_dataframe(experiment_output_path)

Loading results from /home/sac086/wisdm_model_personalization/results/experiment_08-31_train_v1_test_v2_random/
26 users loaded, 28 users excluded
These users may not have had enough data:
['1097', '1205', '1247', '1269', '1276', '1277', '1280', '1480', '1491', '1511', '1518', '1531', '1554', '1679', '1683', '1696', '1723', '1724', '1726', '1745', '1750', '1757', '1761', '1763', '1797', '1802', '1813', '1814']


In [152]:
fig = viz.accuracy_means_by_training_size()
iplot(fig)

In [153]:
fig = viz.plotUserBests()
iplot(fig)

# Trained and tested on lab collected data

In [109]:
experiment_output_path = '/home/sac086/wisdm_model_personalization/results/

'/home/sac086/wisdm_model_personalization/results/experiment_08-31_train_v1_test_v2_random/'

In [ ]:
experim
results_df = viz.get_results_dataframe(experiment_output_path)